In [1]:
import sys
sys.path.append("../../../experiments/") # use utils

import utils
import importlib
importlib.reload(utils)

from utils import calc_auc, get_abstract, normalize_text

import requests
import json
from progressbar import progressbar as pb
from copy import deepcopy

In [2]:
with open("../../../data/doc_id2abstract.json") as f:
    doc_id2abstract = json.load(f)
len(doc_id2abstract)

9476

In [3]:
df = pd.read_csv("../../../data/data_to_process.csv")
print(len(df))
df.head(3)

113


,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


In [28]:
url_template = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=<QUERY>+AND+humans[MeSH%20Terms]&retmode=json"

In [29]:
def reform_query(q):
    tokens = q.split()
    return "[TIAB]+AND+".join(tokens) + "[TIAB]"

## Keywords

In [30]:
for_dataframe = {item: [] for item in ["query_id", "data_source", "pubmed_id"]}

for _, row in pb(df.iterrows(), max_value=len(df)):
    query_id = row.query_id
    data_source = row.data_source
    query = row.query
    query = normalize_text(query, need_lemmatize=False)
    
    new_url = deepcopy(url_template)
    new_url = new_url.replace("<QUERY>", reform_query(query))
    response = requests.get(new_url, headers={'Content-Type': 'application/json'}).json()
    
    id_list = response['esearchresult']['idlist'][:10] # TOP 10 ABSTRACTS
    
    for doc_id in id_list:
        for_dataframe['query_id'].append(query_id)
        for_dataframe['data_source'].append(data_source)
        for_dataframe['pubmed_id'].append(doc_id)

100% (50 of 50) |########################| Elapsed Time: 0:01:07 Time:  0:01:07


In [ ]:
fin_df = (
    pd.DataFrame(for_dataframe)
    .merge(df, on=["query_id", "data_source"])
)

In [ ]:
fin_df['abstract'] = fin_df.pubmed_id.map(doc_id2abstract)
fin_df = fin_df.dropna().query("abstract != ''")
fin_df

In [36]:
fin_df.to_csv("../../../data/pubmed_search/pubmed_search_abstracts_keywords.csv", index=0)

## Question

In [13]:
url_template = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=<QUERY>&retmode=json"

In [14]:
def reform_query(q):
    tokens = q.split()
    return " ".join(tokens)

In [15]:
for_dataframe = {item: [] for item in ["query_id", "data_source", "pubmed_id"]}

for _, row in pb(df.iterrows(), max_value=len(df)):
    query_id = row.query_id
    data_source = row.data_source
    query = row.description
    query = normalize_text(query, need_lemmatize=False).strip()
    
    new_url = deepcopy(url_template)
    new_url = new_url.replace("<QUERY>", reform_query(query))
    response = requests.get(new_url, headers={'Content-Type': 'application/json'}).json()
    
    id_list = response['esearchresult']['idlist'][:10] # TOP 10 ABSTRACTS
    
    for doc_id in id_list:
        for_dataframe['query_id'].append(query_id)
        for_dataframe['data_source'].append(data_source)
        for_dataframe['pubmed_id'].append(doc_id)

100% (50 of 50) |########################| Elapsed Time: 0:01:06 Time:  0:01:06


In [ ]:
fin_df = (
    pd.DataFrame(for_dataframe)
    .merge(df, on=["query_id", "data_source"])


In [ ]:
fin_df['abstract'] = fin_df.pubmed_id.map(doc_id2abstract)
fin_df = fin_df.dropna().query("abstract != ''")

In [26]:
fin_df.to_csv("../../../data/pubmed_search/pubmed_search_abstracts_question.csv", index=0)